In [2]:
# Tiny shakespeare dataset
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

In [3]:
print(len(text))

1115394


In [4]:
# Collect vocabulary of dataset
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


In [5]:
# Tokenise the vocabulary (get the index form for every element of vocab)
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s]
decode = lambda l: ''.join([itos[i] for i in l])

print(encode("hii there"))
print(decode(encode("hii there")))

[46, 47, 47, 1, 58, 46, 43, 56, 43]
hii there


In [6]:
# Load tokenised dataset into a tensor
import torch 
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)
print(data[:1000])

torch.Size([1115394]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59,  1, 39, 56, 43,  1, 39, 50, 50,
         1, 56, 43, 57, 53, 50, 60, 43, 42,  1, 56, 39, 58, 46, 43, 56,  1, 58,
        53,  1, 42, 47, 43,  1, 58, 46, 39, 52,  1, 58, 53,  1, 44, 39, 51, 47,
        57, 46, 12,  0,  0, 13, 50, 50, 10,  0, 30, 43, 57, 53, 50, 60, 43, 42,
         8,  1, 56, 43, 57, 53, 50, 60, 43, 42,  8,  0,  0, 18, 47, 56, 57, 58,
         1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 18, 47, 56, 57, 58,  6,  1, 63,
        53, 59,  1, 49, 52, 53, 61,  1, 15, 39, 47, 59, 57,  1, 25, 39, 56, 41,
      

In [7]:
# Split into train and validation sets
n = int(0.9*len(data))
train_data = data[:n]
val_data = data[n:]

In [8]:
# Define the maximum context length that is used to predict next token
block_size = 8
train_data[:block_size+1]

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

In [9]:
x = train_data[:block_size]
y = train_data[1:block_size+1]
for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print(f"{context} -> {target}")

tensor([18]) -> 47
tensor([18, 47]) -> 56
tensor([18, 47, 56]) -> 57
tensor([18, 47, 56, 57]) -> 58
tensor([18, 47, 56, 57, 58]) -> 1
tensor([18, 47, 56, 57, 58,  1]) -> 15
tensor([18, 47, 56, 57, 58,  1, 15]) -> 47
tensor([18, 47, 56, 57, 58,  1, 15, 47]) -> 58


In [10]:
torch.manual_seed(1337)
batch_size = 4 # total number of independent sequences to process simultaneously
block_size = 8 # context length to predict next token

def get_batch(split):
    data = train_data if split == "train" else val_data
    # random starting indexes

    # to ensure we have enough chars in each sequence
    max_pos = len(data) - block_size 
    # gen batch_size no. ints up to max_pos
    sample_places = torch.randint(max_pos, (batch_size,)) 

    # stack the examples in batch and output (examples right shifted by 1)
    x = torch.stack([data[sp:sp+block_size] for sp in sample_places])
    y = torch.stack([data[(sp+1):(sp+block_size+1)] for sp in sample_places])
    return x, y

xb, yb = get_batch('train')
print('inputs', xb.shape, "\n", xb)
print('outputs', yb.shape, "\n", yb)
print("=== === ===")

for b in range(batch_size):
    for t in range(block_size):
        context = xb[b, :t+1]
        target = yb[b, t]
        print(f"{context} -> {target}")

inputs torch.Size([4, 8]) 
 tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])
outputs torch.Size([4, 8]) 
 tensor([[43, 58,  5, 57,  1, 46, 43, 39],
        [53, 56,  1, 58, 46, 39, 58,  1],
        [58,  1, 58, 46, 39, 58,  1, 46],
        [17, 27, 10,  0, 21,  1, 54, 39]])
=== === ===
tensor([24]) -> 43
tensor([24, 43]) -> 58
tensor([24, 43, 58]) -> 5
tensor([24, 43, 58,  5]) -> 57
tensor([24, 43, 58,  5, 57]) -> 1
tensor([24, 43, 58,  5, 57,  1]) -> 46
tensor([24, 43, 58,  5, 57,  1, 46]) -> 43
tensor([24, 43, 58,  5, 57,  1, 46, 43]) -> 39
tensor([44]) -> 53
tensor([44, 53]) -> 56
tensor([44, 53, 56]) -> 1
tensor([44, 53, 56,  1]) -> 58
tensor([44, 53, 56,  1, 58]) -> 46
tensor([44, 53, 56,  1, 58, 46]) -> 39
tensor([44, 53, 56,  1, 58, 46, 39]) -> 58
tensor([44, 53, 56,  1, 58, 46, 39, 58]) -> 1
tensor([52]) -> 58
tensor([52, 58]) -> 1
tensor([52, 58,  1]) -> 58
t

In [11]:
print(xb) # our input to the transformer

tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])


In [12]:
import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)

class BigramLanguageModel(nn.Module):

    def __init__(self, vocab_size):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table (with trainable weights - 1 linear layer)
        # this was a massive source of confusion, but essentially its equivalent to
        # the neural network bigram model made in Karparthy's earlier video in the same series
        # the number of channels = vocab_size = logits (pre-exponential probabilities) of picking next char given the previous char - bigram
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, idx, targets=None):

        # idx and targets are both (B,T) tensor of integers
        # gets the probabilities C for all next chars in vocab size, given the identity of the current char
        logits = self.token_embedding_table(idx) # (B,T,C)
        
        # if using forward pass without train step
        # then loss = None
        # else perform cross entropy loss on the logit distributions correctly identifying the next target
        # loss implicitly sets and calculates the backpropagation gradients for all the weights in the token_embedding table
        # based on the mathematical calculations performed on the input current characters to get the output (targets) and loss
        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss
    
    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # get the predictions
            logits, loss = self(idx)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

m = BigramLanguageModel(vocab_size)
logits, loss = m(xb, yb)

# the predictions of probailities (in logit format) for each of the 32 input character priors
print(logits.shape)
# the degree to which the logit probabilities had the wrong non one-hot outputs
print(loss)

# represent its predictions as the characters generated based on these logits when treated as probabilities
print(decode(m.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=100)[0].tolist()))

torch.Size([32, 65])
tensor(4.8786, grad_fn=<NllLossBackward0>)

SKIcLT;AcELMoTbvZv C?nq-QE33:CJqkOKH-q;:la!oiywkHjgChzbQ?u!3bLIgwevmyFJGUGp
wnYWmnxKWWev-tDqXErVKLgJ
